# Q-Learning in GridWorld

This notebook demonstrates how Q-learning, a type of reinforcement learning algorithm, can be used to train an agent to navigate a grid-based environment called GridWorld. The agent's objective is to find an optimal path from the start state to the goal state while avoiding obstacles. The agent learns by interacting with the environment and updating its knowledge base, represented by a Q-table, based on the rewards received for its actions.

In [1]:
import numpy as np
import random

## Define the GridWorld Environment with Random Obstacles
In this cell, we define the `GridWorld` class that simulates the environment. The grid has a start state at the top-left corner, a goal state at the bottom-right corner, and random obstacles. The agent can move up, down, left, or right.

In [2]:
class GridWorld:
    def __init__(self, grid_size=(50, 50), num_obstacles=100):
        self.grid_size = grid_size
        self.num_obstacles = num_obstacles
        self.state = (0, 0)  # Start state
        self.goal_state = (grid_size[0]-1, grid_size[1]-1)  # Goal state at the bottom-right corner
        self.actions = ['up', 'down', 'left', 'right']
        self.rewards = np.zeros(grid_size)
        self.rewards[self.goal_state] = 1  # Reward for reaching the goal
        self.obstacle_states = self._place_obstacles()
        for obstacle in self.obstacle_states:
            self.rewards[obstacle] = -1  # Penalty for hitting the obstacle

    def _place_obstacles(self):
        obstacles = set()
        while len(obstacles) < self.num_obstacles:
            obstacle = (random.randint(0, self.grid_size[0]-1), random.randint(0, self.grid_size[1]-1))
            if obstacle != self.state and obstacle != self.goal_state:
                obstacles.add(obstacle)
        return list(obstacles)

    def reset(self):
        self.state = (0, 0)
        return self.state

    def step(self, action):
        next_state = list(self.state)
        if action == 0:  # Up
            next_state[0] = max(0, self.state[0] - 1)
        elif action == 1:  # Down
            next_state[0] = min(self.grid_size[0] - 1, self.state[0] + 1)
        elif action == 2:  # Left
            next_state[1] = max(0, self.state[1] - 1)
        elif action == 3:  # Right
            next_state[1] = min(self.grid_size[1] - 1, self.state[1] + 1)
        
        self.state = tuple(next_state)
        reward = self.rewards[self.state]
        done = self.state == self.goal_state
        return self.state, reward, done

    def print_grid(self):
        grid = np.full(self.grid_size, ' ')
        grid[self.state] = 'S'
        grid[self.goal_state] = 'G'
        for obstacle in self.obstacle_states:
            grid[obstacle] = 'X'
        
        print('\n'.join(' '.join(row) for row in grid))

## Introduction to Q-Learning

Q-learning is a model-free reinforcement learning algorithm where an agent learns the value (Q-value) of taking certain actions in given states. The agent aims to learn an optimal policy that maximizes the expected sum of rewards over time. The Q-values are updated iteratively based on the rewards received for each action using the Bellman equation:

\[ Q(s, a) = Q(s, a) + \alpha \left[ r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right] \]

where:
- \( \alpha \) is the learning rate.
- \( \gamma \) is the discount factor.
- \( r \) is the reward received after taking action \( a \) in state \( s \).
- \( s' \) is the next state after taking action \( a \).
- \( \max_{a'} Q(s', a') \) is the maximum Q-value for all possible actions \( a' \) in the next state \( s' \).

## Define the Q-Learning Algorithm

In this cell, we implement the Q-learning algorithm. The agent will learn to navigate the grid by exploring different actions and updating the Q-values based on the rewards received. The algorithm uses an epsilon-greedy policy to balance exploration (random actions) and exploitation (choosing the best-known action).

In [3]:
def q_learning(env, num_episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    q_table = np.zeros((*env.grid_size, len(env.actions)))
    
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        
        while not done:
            if random.uniform(0, 1) < epsilon:
                action = random.choice(range(len(env.actions)))  # Explore: random action
            else:
                action = np.argmax(q_table[state])  # Exploit: best action based on Q-table
            
            next_state, reward, done = env.step(action)
            
            old_value = q_table[state][action]
            next_max = np.max(q_table[next_state])
            
            # Update Q-value using the Bellman equation
            q_table[state][action] = old_value + alpha * (reward + gamma * next_max - old_value)
            
            state = next_state
    
    return q_table

## Initialize the Environment and Run Q-Learning
We initialize the environment with a 8x8 grid and 16 random obstacles. We then run the Q-learning algorithm to train the agent. The Q-table is printed to show the learned values.

In [4]:
# Initialize environment with a 8x8 grid and 16 obstacles
grid_size = (5, 5)
num_obstacles = 5
env = GridWorld(grid_size, num_obstacles)

# Print the initial grid
print("Initial Grid:")
env.print_grid()

# Run Q-learning
q_table = q_learning(env)

# Print the Q-table
print("Q-Table:")
print(q_table)

Initial Grid:
S   X    
    X   X
         
        X
    X   G
Q-Table:
[[[ 0.37111501  0.36239623  0.39950067  0.4782969 ]
  [ 0.43903665  0.531441    0.4134855  -0.61418559]
  [-0.95788924 -0.9941375   0.4707156   0.        ]
  [ 0.          0.         -0.83376539  0.        ]
  [ 0.         -1.          0.          0.        ]]

 [[ 0.08178877  0.          0.05854693  0.53043491]
  [ 0.45907214  0.59049     0.36816678 -0.45327411]
  [-0.99995502  0.65209693  0.14392283  0.        ]
  [ 0.          0.05896304 -0.97103386 -0.99999985]
  [ 0.          0.          0.         -0.9991405 ]]

 [[ 0.43138201  0.          0.          0.        ]
  [ 0.50440949  0.46078981  0.27653242  0.6561    ]
  [-0.53223531  0.729       0.58460229  0.54901241]
  [ 0.          0.1539      0.65572999  0.03583722]
  [-0.40951    -0.40951     0.46412155  0.0391123 ]]

 [[ 0.07676482  0.          0.          0.        ]
  [ 0.58843878  0.          0.00545716  0.13731516]
  [ 0.63468436 -0.19048993  0.4806741

## Extract and Visualize the Policy

After training, we extract the policy from the Q-table by selecting the action with the highest Q-value for each state. We visualize the policy using symbols to represent each action (up, down, left, right).

In [5]:
# Extract policy
policy = np.argmax(q_table, axis=2)
policy_symbols = {0: '↑', 1: '↓', 2: '←', 3: '→'}

print("\nPolicy:")
for row in policy:
    print(' '.join(policy_symbols[action] for action in row))


Policy:
→ ↓ ← ↑ ↑
→ ↓ ↓ ↓ ↑
↑ → ↓ ← ←
↑ ↑ → ↓ ↑
↑ ↑ → → ↑


## Demonstrate the Learned Policy

We demonstrate the learned policy by showing the agent's path from the start state to the goal state. This helps visualize how the agent navigates the grid based on the learned Q-values.

In [6]:
# Example usage of the learned policy
for _ in range(5):
    state = env.reset()
    path = [state]
    done = False
    while not done:
        action = np.argmax(q_table[state])
        next_state, reward, done = env.step(action)
        path.append(next_state)
        state = next_state
    print(f"Path taken: {path}")
    print("Goal reached!\n")

Path taken: [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4)]
Goal reached!

Path taken: [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4)]
Goal reached!

Path taken: [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4)]
Goal reached!

Path taken: [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4)]
Goal reached!

Path taken: [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4)]
Goal reached!

